In [36]:
import cv2
import itertools
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, Flatten, Activation, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model

## Load the data dir

In [30]:
# non segmented data
data_dir = "../input/plant-seedlings-classification/"
train_dir = os.path.join(data_dir, "train")
data_dir = "../input/plant-seedlings-classification/"
test_dir = os.path.join(data_dir, "test")

In [ ]:
# # segmented data
# data_dir = "../input/plant-seedling-segmented/plant-seedling-segmented/"
# train_dir = os.path.join(data_dir, "seg_train")
# data_dir = "../input/plant-seedling-segmented/plant-seedling-segmented/"
# test_dir = os.path.join(data_dir, "seg_test")

## Image pre-processing: Segmentation

In [28]:
# https://www.kaggle.com/gaborvecsei/plant-seedlings-fun-with-computer-vision/notebook

import cv2
import numpy as np

def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

## Load images and labels

In [31]:
# load train images
images, labels = [], []

for class_folder_name in os.listdir(train_dir):
    class_folder_path = os.path.join(train_dir, class_folder_name)
    
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_128 = cv2.resize(image, (128,128))
        images.append(image_128)
        labels.append(class_folder_name)

images = np.array(images)
labels = np.array(labels)

In [ ]:
# load non segmented
images_test = []

for image_path in glob(os.path.join(test_dir, "*.png")):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image_128 = cv2.resize(image, (128,128))
    images_test.append(image_128)  

images_test = np.array(images_test)

In [ ]:
# # load segmented
# images_test = []

# for image_path in glob(os.path.join(test_dir, "*.png")):
#     image = cv2.imread(image_path, cv2.IMREAD_COLOR)
#     image_299 = cv2.resize(image, (299, 299))
#     image_segmented = segment_plant(image_299)
#     image_sharpened = sharpen_image(image_segmented)
#     images_test.append(image_sharpened)  

# images_test = np.array(images_test)

## Define runtime params

In [ ]:
BATCH_SIZE = 16
EPOCHS = 175
RANDOM_STATE = 11

## Label pre-processing

In [ ]:
# convert the text labels to numerical ones (dict)
text_label_to_num_label_dict = {v:i for i,v in enumerate(np.unique(labels))}
text_label_to_num_label_dict

In [ ]:
# convert numerical ids to text labels (dict)
num_label_to_text_label_dict = {v: k for k, v in text_label_to_num_label_dict.items()}
num_label_to_text_label_dict

In [ ]:
# convert the text labels to numerical ones (list)
truth_labels = np.array([text_label_to_num_label_dict[x] for x in labels])
truth_labels

## CNN model

### 1. Define model architecture

In [32]:
def create_model():
    image = Input(shape=(128, 128, 3))

    conv_1 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')(image)
    bn_1 = BatchNormalization(axis=3)(conv_1)
    act_1 = LeakyReLU(1/10)(bn_1)
    
    conv_2 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu')(act_1)
    bn_2 = BatchNormalization(axis=3)(conv_2)
    act_2 = LeakyReLU(1/10)(bn_2)
    
    pool_1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(act_2)
    
    
    conv_4 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu')(pool_1)
    bn_4 = BatchNormalization(axis=3)(conv_4)
    act_4 = LeakyReLU(1/10)(bn_4)
    
    conv_5 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu')(act_4)
    bn_5 = BatchNormalization(axis=3)(conv_5)
    act_5 = LeakyReLU(1/10)(bn_5)
    
    pool_2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(act_5)
    

    conv_7 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu')(pool_2)
    bn_7 = BatchNormalization(axis=3)(conv_7)
    act_7 = LeakyReLU(1/10)(bn_7)
    
    conv_8 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu')(act_7)
    bn_8 = BatchNormalization(axis=3)(conv_8)
    act_8 = LeakyReLU(1/10)(bn_8)
    
    pool_3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(act_8)
    

    flat_1 = Flatten()(pool_3)
    
    drop_1 = Dropout(0.1)(flat_1)
    dense_1 = Dense(128)(drop_1)
    batch_n_1 = BatchNormalization(axis=-1)(dense_1)
    a_1 = Activation(activation='tanh')(batch_n_1)
    
    drop_3 = Dropout(0.1)(a_1)
    dense_3 = Dense(12)(drop_3)
    batch_n_3 = BatchNormalization(axis=-1)(dense_3)
    a_3 = Activation(activation='softmax')(batch_n_3)
    
    model = Model(inputs=image, outputs=a_3)
    
    adam_apt = keras.optimizers.Adam(lr=0.002, beta_1=0.99, beta_2=0.99, epsilon=1e-08)
    
    model.compile(loss='sparse_categorical_crossentropy',
                   optimizer=adam_apt,
                   metrics=['accuracy'])
    model.summary()
    return model

### 2. Define model callbacks

In [ ]:
my_callbacks = [
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=1e-5, patience=10, verbose=1),
    ModelCheckpoint('model_best.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
]

### 3. Train Test split (80-20)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(images,
                                                    truth_labels,
                                                    shuffle=True,
                                                    train_size=0.8,
                                                    random_state=RANDOM_STATE
                                                    )

### 4. Image augmentation

In [ ]:
gen = ImageDataGenerator(
        rotation_range=360.,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.3,
        height_shift_range=0.3
)

### 5. Create model and train

In [ ]:
model_1 = create_model()
history = model_1.fit_generator(gen.flow(x_train, y_train,batch_size=BATCH_SIZE),
           epochs=EPOCHS,
           verbose=1,
           shuffle=True,
           validation_data=(x_val, y_val),
           callbacks=my_callbacks
                               )

In [38]:
# printing model summary of the saved model
plot_model(model_1, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### 6. Plot accuracy and loss graphs

In [ ]:
plt.plot(history.history['accuracy'], label= 'train')
plt.plot(history.history['val_accuracy'], label = 'val')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'val')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

### 7. Load best model and predict on it

In [ ]:
# load model
model_best = load_model('model_best.h5')

In [ ]:
# get predictions on test data
prob = model_best.predict(images_test, verbose=1)
pred = prob.argmax(axis=-1)

In [ ]:
# map the number labels back to the text labels
text_predictions = []
for i in pred:
    text_predictions.append(num_label_to_text_label_dict[i])

In [ ]:
# save to df
df = pd.DataFrame({'file': os.listdir('../input/plant-seedlings-classification/test'), 'species': text_predictions})
df.to_csv('model_results.csv', index=False)

In [ ]:
# # save to df
# df = pd.DataFrame({'file': os.listdir('../input/plant-seedling-segmented/plant-seedling-segmented/seg_test'), 'species': text_predictions})
# df.to_csv('model_results.csv', index=False)

### 8. Confusion matrix for evaluation data

In [ ]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=False):
    if cmap is None:
        cmap = plt.get_cmap('Oranges')

    plt.figure(figsize=(12, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylim(len(target_names)-0.5, -0.5)
    plt.ylabel('True labels')
    plt.xlabel('Predicted labels')
    plt.savefig(title + '.png', dpi=500, bbox_inches = 'tight')
    plt.show()

In [ ]:
# get predictions on eval dataset
y_predict=model_best.predict(x_val)
y_pred = y_predict.argmax(axis=-1)

In [ ]:
# get list of text labels
text_label_list = []
for i in num_label_to_text_label_dict:
    text_label_list.append(num_label_to_text_label_dict[i])
text_label_list

In [ ]:
# call the confusion_matrix function to build a confusion matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)

In [ ]:
# print the cm in a pretty format
plot_confusion_matrix(cm, text_label_list, "CNN model Confusion Matrix")